# 1. Import Required Lib

In [2]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from bleu_score import BLEU_SCORE
# from models_viet import EncoderRNN, AttentionDecoderRNN, DecoderRNN
from load_dataset_viet import *
# from define_training_viet import *
import torchtext
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline
import math
import copy
import seaborn

In [2]:
device = torch.device('cuda')

# 2. Build data loaders

In [3]:
class Vietnamese(Dataset):
    def __init__(self, df, val = False):
        self.df = df
        self.val = val
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx,:]['en_idized']
        viet = self.df.iloc[idx,:]['vi_idized']
        en_len = self.df.iloc[idx,:]['en_len']
        vi_len = self.df.iloc[idx,:]['vi_len']
        if self.val:
            en_data = self.df.iloc[idx,:]['en_data'].lower()
            return [viet,english,vi_len,en_len,en_data]
        else:
            return [viet,english,vi_len,en_len]
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
def vocab_collate_func(batch):
    MAX_LEN_EN = 48
    MAX_LEN_VI = 48
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []
    for datum in batch:
        en_len.append(datum[3])
        vi_len.append(datum[2])
    max_batch_length_en = max(en_len)
    max_batch_length_vi = max(vi_len)
    if max_batch_length_en < MAX_LEN_EN:
        MAX_LEN_EN = max_batch_length_en
    if max_batch_length_vi < MAX_LEN_VI:
        MAX_LEN_VI = max_batch_length_vi
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN_VI:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN_VI]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN_VI - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN_EN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN_EN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN_EN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s2)
        vi_data.append(padded_vec_s1)
    vi_data = np.array(vi_data)
    en_data = np.array(en_data)
    vi_len = np.array(vi_len)
    en_len = np.array(en_len)
#     sorted_vi_len = np.argsort(-vi_len)
#     vi_data = vi_data[sorted_vi_len]
#     en_data = en_data[sorted_vi_len]
#     vi_len = vi_len[sorted_vi_len]
#     en_len = en_len[sorted_vi_len]
#     print(en_len)
    vi_len[vi_len>MAX_LEN_VI] = MAX_LEN_VI
    en_len[en_len>MAX_LEN_EN] = MAX_LEN_EN
        
    return [torch.from_numpy(vi_data), torch.from_numpy(en_data),
            torch.from_numpy(vi_len), torch.from_numpy(en_len)]

def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

def convert_id_list_2_sent(list_idx, lang_obj):
    word_list = []
    if type(list_idx) == list:
        for i in list_idx:
            if i not in set([EOS_token]):
                word_list.append(lang_obj.index2word[i])
    else:
        for i in list_idx:
            if i.item() not in set([EOS_token,SOS_token,PAD_IDX]):
                word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [4]:
def vocab_collate_func_val(batch):
    return [torch.from_numpy(np.array(batch[0][0])).unsqueeze(0), torch.from_numpy(np.array(batch[0][1])).unsqueeze(0),
            torch.from_numpy(np.array(batch[0][2])).unsqueeze(0), torch.from_numpy(np.array(batch[0][3])).unsqueeze(0),batch[0][4]]

In [5]:
# MAX_LEN = 48
train,val,test,en_lang,vi_lang = train_val_load(48, "", '/scratch/ark576/machine_translation_data/')

In [6]:
def shuffle_sorted_batches(train, bs):
    batch_samp_list = list(BatchSampler(SequentialSampler(train), bs, drop_last = False))
    np.random.shuffle(batch_samp_list)
    batch_samp_list_merged = list(itertools.chain(*batch_samp_list))
    return train.iloc[batch_samp_list_merged,:]

In [7]:
train.head()

,en_data,vi_data,en_tokenized,vi_tokenized,en_idized,vi_idized,en_len,vi_len
0,Rachel Pike : The science behind a climate hea...,Khoa_học đằng_sau một tiêu_đề về khí_hậu,"[rachel, pike, :, the, science, behind, a, cli...","[khoa_học, đằng_sau, một, tiêu_đề, về, khí_hậu]","[8225, 2, 137, 4, 150, 573, 9, 15, 3717, 1]","[7, 1433, 5, 3054, 4, 13, 1]",10,7
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên_gia hoá_học khí_quyển Ra...","[in, 4, minutes, ,, atmospheric, chemist, rach...","[trong, 4, phút, ,, chuyên_gia, hoá_học, khí_q...","[12, 5080, 264, 7, 20, 4552, 8225, 2, 3006, 9,...","[23, 812, 253, 8, 1043, 1342, 20, 8510, 2, 161...",51,57
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to_lớn của nhữ...,"[i, &apos;d, like, to, talk, to, you, today, a...","[tôi, muốn, cho, các, bạn, biết, về, sự, to_lớ...","[23, 247, 16, 13, 272, 13, 33, 273, 100, 4, 14...","[26, 152, 47, 11, 38, 149, 4, 92, 787, 9, 15, ...",28,26
3,Headlines that look like this when they have t...,Có những dòng trông như thế_này khi bàn về biế...,"[headlines, that, look, like, this, when, they...","[có, những, dòng, trông, như, thế_này, khi, bà...","[6, 11, 24, 16, 21, 49, 35, 31, 13, 36, 51, 15...","[21, 15, 530, 419, 17, 193, 34, 488, 4, 1227, ...",32,25
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh_vực ...,"[they, are, both, two, branches, of, the, same...","[cả, hai, đều, là, một, nhánh, của, cùng, một,...","[35, 55, 402, 118, 3364, 5, 4, 154, 260, 5, 20...","[41, 273, 70, 16, 5, 3618, 9, 178, 5, 198, 23,...",14,16


In [8]:
bs_dict = {'train':128,'validate':1, 'train_val':1,'val_train':128, 'test':1}
shuffle_dict = {'train':True,'validate':False, 'train_val':False,'val_train':True, 'test':False}
# train_used = shuffle_sorted_batches(train_sorted, bs_dict['train'])
# train_used = train.iloc[:50]
train_used = train
val_used = val
# val_used = val.iloc[:20]
collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val,\
                   'train_val':vocab_collate_func_val,'val_train':vocab_collate_func,'test': vocab_collate_func_val}
transformed_dataset = {'train': Vietnamese(train_used),
                       'validate': Vietnamese(val_used, val = True),
                       'train_val':Vietnamese(train.iloc[:50], val = True),
                       'val_train':Vietnamese(val_used),
                       'test':Vietnamese(test, val= True)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
                    shuffle=shuffle_dict[x], num_workers=0) for x in ['train', 'validate', 'train_val','val_train', 'test']}

# 3. Build Models

In [3]:
class EncoderCNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size,n_layers, device = 'cuda'):
        super(EncoderCNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.embedding = Embedding(input_size, embed_dim,PAD_IDX)
        self.embedding_pos = Embedding(1024, embed_dim,999)
        nn.init.uniform_(self.embedding_pos.weight, -0.05, 0.05)
        self.dropout_in = nn.Dropout(p = 0.5)
        self.n_layers = n_layers
        self.kernel_size = 3
        self.hids_in_C = [embed_dim]+[hidden_size]*(n_layers-1)
        self.hids_out_C = [hidden_size]*n_layers
        self.hids_in_A = [embed_dim]+[hidden_size]*(3*n_layers-1)
        self.hids_out_A = [hidden_size]*(3*n_layers)
        self.convsA = nn.ModuleList()
        self.convsC = nn.ModuleList()
        self.device = device
        for i in range(len(self.hids_out_A)):
            self.convsA.append(nn.Conv1d(in_channels=self.hids_in_A[i],out_channels=self.hids_out_A[i],
                                        kernel_size=self.kernel_size, padding=self.kernel_size//2))
        for i in range(len(self.hids_out_C)):
            self.convsC.append(nn.Conv1d(in_channels=self.hids_in_C[i],out_channels=self.hids_out_C[i],
                                        kernel_size=self.kernel_size, padding=self.kernel_size//2))
            
        for layer in self.convsA:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(layer.weight)
            nn.init.uniform_(layer.weight,
                             (self.kernel_size * fan_in)**(-0.5) * -1,
                             (self.kernel_size * fan_in)**(-0.5))

        for layer in self.convsC:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(layer.weight)
            nn.init.uniform_(layer.weight,
                             (self.kernel_size * fan_in)**(-0.5) * -1,
                             (self.kernel_size * fan_in)**(-0.5))

        # scale gradients
        for layer in self.convsA:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(layer.weight)
            layer.weight.register_hook(
                lambda grad: grad / math.sqrt(fan_in))
            layer.bias.register_hook(
                lambda grad: grad / math.sqrt(fan_in))

        for layer in self.convsC:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(layer.weight)
            layer.weight.register_hook(
                lambda grad: grad / math.sqrt(fan_in))
            layer.bias.register_hook(
                lambda grad: grad / math.sqrt(fan_in))
            
            
    def forward(self, enc_inp, src_len):
        batch_size, seqlen = enc_inp.size()
        out = self.embedding(enc_inp)
        pad_mask = enc_inp.eq(PAD_IDX)
        pos_seq = torch.zeros_like(enc_inp)
        for i in range(batch_size):
            pos_seq[i] = torch.arange(0, seqlen)
        pos_seq.masked_fill_(pad_mask, 999)
        pos_out = self.embedding_pos(pos_seq)
        x = out+pos_out
        x_input = self.dropout_in(x.transpose(1, 2))
        a = x_input
        resid_a = torch.zeros([batch_size, self.hidden_size, seqlen]).to(self.device)
        resid_c = torch.zeros([batch_size, self.hidden_size, seqlen]).to(self.device)
        resids_a = [resid_a]
        for op in self.convsA:
            resid_a = resids_a[-1]
            a = torch.tanh(op(self.dropout_in(a)))
            a = a+resid_a
            resids_a.append(a)
            
        final_out_A = a.transpose(2,1)
    
        c = x_input
        resids_c = [resid_c]
        for op in self.convsC:
            resid_c = resids_c[-1]
            c = torch.tanh(op(self.dropout_in(c)))
            c = c+resid_c
            resids_c.append(c)
        final_out_C = c.transpose(2,1)
        return (final_out_A, final_out_C)

In [2]:
class Attention_Module(nn.Module):
    def __init__(self, hidden_dim, output_dim, device = 'cuda'):
        super(Attention_Module, self).__init__()
        self.l1 = Linear(hidden_dim, output_dim, bias = False)
        self.l2 = Linear(hidden_dim+output_dim, output_dim, bias =  False)
        self.device = device
        
    def forward(self, hidden, encoder_outs, src_lens):
        ''' hiddden: bsz x hidden_dim
        encoder_outs: bsz x sq_len x encoder dim (output_dim)
        src_lens: bsz
        
        x: bsz x output_dim
        attn_score: bsz x sq_len'''
        x = self.l1(hidden)
        att_score = (encoder_outs.transpose(0,1) * x.unsqueeze(0)).sum(dim = 2)
        seq_mask = sequence_mask(src_lens, max_len = max(src_lens).item(), device = self.device).transpose(0,1)
        masked_att = seq_mask*att_score
        masked_att[masked_att==0] = -1e10
        attn_scores = F.softmax(masked_att, dim=0)
        x = (attn_scores.unsqueeze(2) * encoder_outs.transpose(0,1)).sum(dim=0)
        x = torch.tanh(self.l2(torch.cat((x, hidden), dim=1)))
        return x, attn_scores
        
class AttentionDecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, n_layers = 1, attention = True, device = 'cuda'):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        encoder_output_size = hidden_size
        self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        self.dropout = nn.Dropout(p=0.1)
        self.n_layers = n_layers
        self.device = device
        self.att_layer = Attention_Module(self.hidden_size, encoder_output_size,self.device) if attention else None
        self.layers = nn.ModuleList([
            LSTMCell(
                input_size=self.hidden_size + embed_dim if ((layer == 0) and attention) else embed_dim if layer == 0 else hidden_size,
                hidden_size=hidden_size,
            )
            for layer in range(self.n_layers)
        ])
        self.fc_out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, dec_input,context_vector, prev_hiddens,prev_cs,encoder_outputs,src_len):
        bsz = dec_input.size(0)
        output = self.embedding(dec_input)
        output = self.dropout(output)
        if self.att_layer is not None:
            cated_input = torch.cat([output.squeeze(1),context_vector], dim = 1)
        else:
            cated_input = output.squeeze(1)
        new_hiddens = []
        new_cs = []
        for i, rnn in enumerate(self.layers):
            hidden, c = rnn(cated_input, (prev_hiddens[i], prev_cs[i]))
            cated_input = self.dropout(hidden)
            new_hiddens.append(hidden.unsqueeze(0))
            new_cs.append(c.unsqueeze(0))
        new_hiddens = torch.cat(new_hiddens, dim = 0)
        new_cs = torch.cat(new_cs, dim = 0)

        # apply attention using the last layer's hidden state
        if self.att_layer is not None:
            out, attn_score = self.att_layer(hidden, encoder_outputs, src_len)
        else:
            out = hidden
            attn_score = None
        context_vec = out
        out = self.dropout(out)
        out_vocab = self.softmax(self.fc_out(out))

        return out_vocab, context_vec, new_hiddens, new_cs, attn_score

# 4. Utilites functions, training and evaluation functions 

In [18]:
def Embedding(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m


def LSTM(input_size, hidden_size, **kwargs):
    m = nn.LSTM(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def Linear(in_features, out_features, bias=True, dropout=0):
    """Linear layer (input: N x T x C)"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

In [19]:
def sequence_mask(sequence_length, max_len=None, device = 'cuda'):
    if max_len is None:
        max_len = sequence_length.max().item()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).repeat([batch_size,1])
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return (seq_range_expand < seq_length_expand).float()

In [22]:
def encode_decode_cnn(encoder,decoder,data_en,data_de,src_len,tar_len,rand_num = 0.95, val = False):
    if not val:
        use_teacher_forcing = True if random.random() < rand_num else False
    #     print("tar_len",tar_len)
        bss = data_en.size(0)
        en_out = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens, prev_cs = decoder.initHidden(bss)
#         n_layers, _, _ = prev_hiddens.size()
#         print(n_layers)
#         prev_hiddens = en_out[1].sum(1).unsqueeze(0).repeat(n_layers,1,1)
#         prev_cs = prev_hiddens
        prev_output = torch.zeros((bss, en_out[0].size(-1))).to(device)
#         prev_output = en_out[1][:,0,:]
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        if use_teacher_forcing:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                decoder_input = data_de[:,i].view(-1,1)
#                 print(decoder_input)
            d_out = torch.cat(d_out,dim=-1)
    #         print("softmax mat", d_out.shape)
        else:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
                d_out.append(out_vocab.unsqueeze(-1))
                topv, topi = out_vocab.topk(1)
                decoder_input = topi.squeeze().detach().view(-1,1)
#                 print(decoder_input)
                decoder_input = topi.squeeze().view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
        return d_out
    else:
        encoder.eval()
        decoder.eval()
        bss = data_en.size(0)
        en_out = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens, prev_cs = decoder.initHidden(bss)
#         prev_hiddens = enc_c.sum(1)
#         prev_cs = prev_hiddens
#         prev_hiddens, prev_cs = decoder.initHidden(bss)
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out[0].size(-1))).to(device)
#         prev_output = en_out[1][:,0,:]
        d_out = []
        for i in range(max_tar_len_batch):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
            d_out.append(out_vocab.unsqueeze(-1))
            topv, topi = out_vocab.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
#             decoder_input = topi.squeeze().view(-1,1)
        d_out = torch.cat(d_out,dim=-1)
        return d_out

In [24]:
def validation_new_cnn(encoder, decoder, val_dataloader, lang_en,m_type, verbose = False, replace_unk =True):
    encoder.eval()
    decoder.eval()
    pred_corpus = []
    true_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    for data in val_dataloader:
        encoder_i = data[0].to(device)
        src_len = data[2].to(device)
        bs,sl = encoder_i.size()[:2]
        en_out = encoder(encoder_i,src_len)
        max_src_len_batch = max(src_len).item()
        prev_hiddens, prev_cs = decoder.initHidden(bs)
#         n_layers, _, _ = prev_hiddens.size()
#         prev_hiddens = en_out[1].sum(1).unsqueeze(0).repeat(n_layers,1,1)
#         prev_cs = prev_hiddens
        decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
#         prev_output = en_out[1][:,0,:]
        prev_output = torch.zeros((bs, en_out[0].size(-1))).to(device)
        d_out = []
        for i in range(sl*2):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len)
            topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
            d_out.append(topi.item())
            decoder_input = topi.squeeze().detach().view(-1,1)
            if topi.item() == EOS_token:
                break
        

#         true_sent = convert_id_list_2_sent(data[1][0],lang_en)
#         true_corpus.append(true_sent)
        if replace_unk:
            true_sent = convert_id_list_2_sent(data[1][0],lang_en)
            true_corpus.append(true_sent)
        else:
            true_corpus.append(data[-1])
        pred_sent = convert_id_list_2_sent(d_out,lang_en)
        pred_corpus.append(pred_sent)
        if verbose:
            print("True Sentence:",data[-1])
            print("Pred Sentence:", pred_sent)
            print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return score

In [26]:
def train_model(encoder_optimizer,decoder_optimizer, encoder, decoder, loss_fun,m_type, dataloader, en_lang,\
                num_epochs=60, val_every = 1, train_bleu_every = 10,clip = 0.1, rm = 0.8, enc_scheduler = None,\
               dec_scheduler = None, enc_dec_fn = encode_decode_cnn, val_fn = validation_new_cnn):
    best_score = 0
    best_bleu = 0
    loss_hist = {'train': [], 'val_train': []}
    bleu_hist = {'train': [], 'validate': []}
    best_encoder_wts = None
    best_decoder_wts = None
    phases = ['train','val_train']
    for epoch in range(num_epochs):
        for ex, phase in enumerate(phases):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train()
                decoder.train()
            else:
                encoder.eval()
                decoder.eval()
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_i = data[0].to(device)
                decoder_i = data[1].to(device)
                src_len = data[2].to(device)
                tar_len = data[3].to(device)
                if phase == 'val_train':  
                    out = enc_dec_fn(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = True)
                else:
                    out = enc_dec_fn(encoder,decoder,encoder_i,decoder_i,src_len,tar_len,rand_num=rm,val = False)
                N = decoder_i.size(0)
                loss = loss_fun(out.float(), decoder_i.long())
                running_loss += loss.item() * N
                
                total += N
                if phase == 'train':
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    
            epoch_loss = running_loss / total 
            loss_hist[phase].append(epoch_loss)
            print("epoch {} {} loss = {}, time = {}".format(epoch, phase, epoch_loss,
                                                                           time.time() - start))
            
        if (enc_scheduler is not None) and (dec_scheduler is not None):
            enc_scheduler.step(epoch_loss)
            dec_scheduler.step(epoch_loss)
        if epoch%val_every == 0:
            val_bleu_score = val_fn(encoder,decoder, dataloader['validate'], en_lang, m_type)
            bleu_hist['validate'].append(val_bleu_score)
            print("validation BLEU = ", val_bleu_score)
            if val_bleu_score > best_bleu:
                best_bleu = val_bleu_score
                best_encoder_wts = encoder.state_dict()
                best_decoder_wts = decoder.state_dict()
        print('='*50)
    encoder.load_state_dict(best_encoder_wts)
    decoder.load_state_dict(best_decoder_wts)
    print("Training completed. Best BLEU is {}".format(best_bleu))
    return encoder,decoder,loss_hist,bleu_hist

In [27]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
seaborn.set_style("darkgrid",{"Droid Sans Fallback":['simhei', 'Arial']})

In [28]:
def draw(att_mat, x, y):
    x = x.split(' ')
    y = y.split(' ')
    x += ['EOS']
    y += ['EOS']
    att_mat = att_mat.data.cpu().squeeze(1).numpy()
    seaborn.heatmap(att_mat, 
                    xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                    cbar=False)

# 5. Without attention (Training and Evaluation)

In [4]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [36]:
enc_cnn = EncoderCNN(vi_lang.n_words, 512,512,2).to(device)
dec_cnn = AttentionDecoderRNN(en_lang.n_words, 512, 512, n_layers=1,attention=True,\
                              device = 'cuda', enc_type = 'conv').to(device)

/home/ark576/.conda/envs/nlp/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [38]:
# encoder_optimizer = optim.Adam(encoder_wo_att.parameters(), lr = 5e-3)
# decoder_optimizer = optim.Adam(decoder_wo_att.parameters(), lr = 5e-3)
encoder_optimizer = optim.SGD(encoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-4,factor = 0.5,  patience=0)
decoder_optimizer = optim.SGD(decoder_wo_att.parameters(), lr=0.25,nesterov=True, momentum = 0.99)
dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-4,factor = 0.5,  patience=0)

In [ ]:
enc_cnn, dec_cnn, loss_hist, acc_hist = train_model(encoder_optimizer, decoder_optimizer, enc_cnn, dec_cnn, criterion,\
                                            "attention", dataloader,en_lang, num_epochs = 150,rm = 0.95,\
                                           enc_scheduler = None, dec_scheduler = None, clip = 0.1, \
                                                    enc_dec_fn = encode_decode_cnn_2, val_fn = validation_new_cnn_2)

In [121]:
torch.save(enc_cnn.state_dict(), 'conv_enc.pth')
torch.save(dec_cnn.state_dict(), 'conv_dec.pth')